In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
data = pd.read_csv('04.reduced_telco_data.csv')
data.head()

,SeniorCitizen,tenure,PhoneService,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,Contract_Month-to-Month,Contract_One year,Contract_Two year,...,DeviceProtection_Yes,TechSupport_No,TechSupport_Unknown,TechSupport_Yes,StreamingTV_No,StreamingTV_Unknown,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_Unknown,StreamingMovies_Yes
0,0,-0.244200,1,1,0.021083,0.001391,0,1,0,0,...,0,1,0,0,0,0,1,0,0,1
1,1,-0.306748,1,0,0.053733,0.002613,0,0,0,1,...,0,1,0,0,1,0,0,1,0,0
2,0,-0.744587,1,0,0.004393,0.000401,0,0,1,0,...,0,1,0,0,1,0,0,1,0,0
3,0,1.069316,1,1,0.020597,0.002886,0,0,0,1,...,0,1,0,0,1,0,0,1,0,0
4,0,-0.619490,1,0,0.041444,0.001535,0,1,0,0,...,1,1,0,0,1,0,0,1,0,0


In [4]:
y = data.iloc[:,6].values
tmp = data.drop(data.columns[6], axis=1)
X = tmp.iloc[:,:].values

In [5]:
X_train,X_test,y_train,y_test = train_test_split(
    X,y,
    test_size=0.2,
    random_state= 42,
    stratify=y,
)

In [6]:
model1 = LogisticRegression(random_state=42, solver='liblinear', C=0.001, max_iter=5000,verbose=1,class_weight='balanced')
model1.fit(X_train, y_train) 


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report,roc_auc_score

model = model1
y_train_pred = model.predict(X_train)
train_accuracy_explicit = accuracy_score(y_train, y_train_pred)
y_test_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred)
recall = recall_score(y_test, y_test_pred)
f1score = f1_score(y_test, y_test_pred)
auc = roc_auc_score(y_test, model.predict_proba(X_test)[:,1])


print(f"Training_Accuracy: {train_accuracy_explicit}")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1score}")
print(classification_report(y_test, y_test_pred))

[LibLinear]Training_Accuracy: 0.7666964285714286
Accuracy: 0.7758035714285715
Precision: 0.7034944237918216
Recall: 0.9014862804878049
F1-Score: 0.7902781257830118
              precision    recall  f1-score   support

           0       0.88      0.66      0.76      5952
           1       0.70      0.90      0.79      5248

    accuracy                           0.78     11200
   macro avg       0.79      0.78      0.77     11200
weighted avg       0.80      0.78      0.77     11200



In [7]:
model2 =  LogisticRegression(random_state=42, solver='saga', C=1000, max_iter=5000,verbose=1,class_weight='balanced',penalty='elasticnet',l1_ratio=0.5)
model2.fit(X_train, y_train) 

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report,roc_auc_score

model = model2
y_train_pred = model.predict(X_train)
train_accuracy_explicit = accuracy_score(y_train, y_train_pred)
y_test_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred)
recall = recall_score(y_test, y_test_pred)
f1score = f1_score(y_test, y_test_pred)
auc = roc_auc_score(y_test, model.predict_proba(X_test)[:,1])


print(f"Training_Accuracy: {train_accuracy_explicit}")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1score}")
print(classification_report(y_test, y_test_pred))

convergence after 102 epochs took 3 seconds
Training_Accuracy: 0.7666964285714286
Accuracy: 0.7758035714285715
Precision: 0.7034944237918216
Recall: 0.9014862804878049
F1-Score: 0.7902781257830118
              precision    recall  f1-score   support

           0       0.88      0.66      0.76      5952
           1       0.70      0.90      0.79      5248

    accuracy                           0.78     11200
   macro avg       0.79      0.78      0.77     11200
weighted avg       0.80      0.78      0.77     11200



[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.1s finished


In [8]:
model3 =  LogisticRegression(random_state=42, solver='saga', C=0.001, max_iter=5000,verbose=1,class_weight='balanced',penalty='elasticnet',l1_ratio=0.5)
model3.fit(X_train, y_train) 

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report,roc_auc_score

model = model3
y_train_pred = model.predict(X_train)
train_accuracy_explicit = accuracy_score(y_train, y_train_pred)
y_test_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred)
recall = recall_score(y_test, y_test_pred)
f1score = f1_score(y_test, y_test_pred)
auc = roc_auc_score(y_test, model.predict_proba(X_test)[:,1])


print(f"Training_Accuracy: {train_accuracy_explicit}")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1score}")
print(classification_report(y_test, y_test_pred))

rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
convergence after 12 epochs took 0 seconds
Training_Accuracy: 0.7666964285714286
Accuracy: 0.7758035714285715
Precision: 0.7034944237918216
Recall: 0.9014862804878049
F1-Score: 0.7902781257830118
              precision    recall  f1-score   support

           0       0.88      0.66      0.76      5952
           1       0.70      0.90      0.79      5248

    accuracy                           0.78     11200
   macro avg       0.79      0.78      0.77     11200
weighted avg       0.80      0.78      0.77     11200



[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s finished


In [9]:
model4 =  LogisticRegression(random_state=42, solver='liblinear', C=1000, max_iter=5000,verbose=1,class_weight='balanced', penalty='l1')
model4.fit(X_train, y_train) 

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report,roc_auc_score

model = model4
y_train_pred = model.predict(X_train)
train_accuracy_explicit = accuracy_score(y_train, y_train_pred)
y_test_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred)
recall = recall_score(y_test, y_test_pred)
f1score = f1_score(y_test, y_test_pred)
auc = roc_auc_score(y_test, model.predict_proba(X_test)[:,1])


print(f"Training_Accuracy: {train_accuracy_explicit}")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1score}")
print(classification_report(y_test, y_test_pred))

[LibLinear]Training_Accuracy: 0.7666964285714286
Accuracy: 0.7758035714285715
Precision: 0.7034944237918216
Recall: 0.9014862804878049
F1-Score: 0.7902781257830118
              precision    recall  f1-score   support

           0       0.88      0.66      0.76      5952
           1       0.70      0.90      0.79      5248

    accuracy                           0.78     11200
   macro avg       0.79      0.78      0.77     11200
weighted avg       0.80      0.78      0.77     11200



In [ ]:
# saving the model
import joblib
import os

# Define the filename and path
model_filename = 'logistic_regression_model.joblib'

# Save the model
joblib.dump(model1, model_filename)

In [ ]:
import joblib
import pandas as pd
from sklearn.metrics import accuracy_score

# 1. Load the model from the file
loaded_model = joblib.load(model_filename)

In [ ]:
def tenure_z_score_calculation (value: float) -> float:
    import pandas as pd
    import pickle
    with open('tenure_zscore_scaler.pkl', 'rb') as f:
        tenure_scaler = pickle.load(f)
    tenure_df = pd.DataFrame({'tenure': [value]})
    final = tenure_scaler.transform(tenure_df)[0][0]
    return final



In [ ]:
def MonthlyCharges_TotalCharges_minmaxscaler_calculation (MonthlyCharges_value: float, TotalCharges_value:float) -> float:
    import pandas as pd
    import pickle
    with open('charges_minmax_scaler.pkl', 'rb') as f:
        charges_scaler = pickle.load(f)
    MonthlyCharges_TotalCharges_df = pd.DataFrame({'MonthlyCharges': [68.61],'TotalCharges': [205.83]})
    MonthlyCharges_final = charges_scaler.transform(MonthlyCharges_TotalCharges_df)[0][0]
    TotalCharges_final = charges_scaler.transform(MonthlyCharges_TotalCharges_df)[0][1]
    return MonthlyCharges_final,TotalCharges_final



In [ ]:
def predict_new_customer_churn(customer_data, model, threshold=0.5):
    import numpy as np
    # Convert input to numpy array
    data_array = np.array(customer_data, dtype=np.float32).reshape(1, -1)

    # Get churn probability (probability of class 1)
    probability = model.predict_proba(data_array)[:, 1][0]

    # Apply threshold
    prediction = 1 if probability >= threshold else 0

    return prediction, probability

In [ ]:
SeniorCitizen=0
tenure=3
PhoneService=1
PaperlessBilling=0
MonthlyCharges=68.61
TotalCharges=205.83
Contract_Month_to_Month=1 
Contract_One_year=0
Contract_Two_year=0 
PaymentMethod_Bank_transfer_automatic=0
PaymentMethod_CreditP_card_automatic=0
PaymentMethod_Electronic_check=0
PaymentMethod_Mailed_check=1
PaymentMethod_Unknown=0
gender_Female=1
gender_Unknown=0  
Partner_No=1
Partner_Unknown=0
Partner_Yes=0
Dependents_No=0
Dependents_Unknown=0
Dependents_Yes=1
MultipleLines_No=0
MultipleLines_Unknown=0
MultipleLines_Yes=1
InternetService_DSL=0
InternetService_Fiber_optic=0
InternetService_No=1
OnlineSecurity_No=1
OnlineSecurity_Unknown=0
OnlineSecurity_Yes=0
OnlineBackup_No=1
OnlineBackup_Unknown=0
OnlineBackup_Yes=0
DeviceProtection_No=1
DeviceProtection_Unknown=0
DeviceProtection_Yes=0
TechSupport_No=1
TechSupport_Unknown=0
TechSupport_Yes=0
StreamingTV_No=1
StreamingTV_Unknown=0
StreamingTV_Yes=0
StreamingMovies_No=1
StreamingMovies_Unknown=0
StreamingMovies_Yes=0


tenure = tenure_z_score_calculation(tenure)
MonthlyCharges,TotalCharges=MonthlyCharges_TotalCharges_minmaxscaler_calculation(68.61,205.83)



In [ ]:
sample_customer_features = [
    SeniorCitizen, tenure, PhoneService, PaperlessBilling, MonthlyCharges, TotalCharges, Contract_Month_to_Month, Contract_One_year, Contract_Two_year, PaymentMethod_Bank_transfer_automatic, PaymentMethod_CreditP_card_automatic, PaymentMethod_Electronic_check, PaymentMethod_Mailed_check, PaymentMethod_Unknown, gender_Female, gender_Unknown, Partner_No, Partner_Unknown, Partner_Yes, Dependents_No, Dependents_Unknown, Dependents_Yes, MultipleLines_No, MultipleLines_Unknown, MultipleLines_Yes, InternetService_DSL, InternetService_Fiber_optic, InternetService_No, OnlineSecurity_No, OnlineSecurity_Unknown, OnlineSecurity_Yes, OnlineBackup_No, OnlineBackup_Unknown, OnlineBackup_Yes, DeviceProtection_No, DeviceProtection_Unknown, DeviceProtection_Yes, TechSupport_No, TechSupport_Unknown, TechSupport_Yes, StreamingTV_No, StreamingTV_Unknown, StreamingTV_Yes, StreamingMovies_No, StreamingMovies_Unknown, StreamingMovies_Yes
]

In [ ]:
# Assuming your model is loaded as 'loaded_model'
prediction, probability = predict_new_customer_churn(
    customer_data=sample_customer_features, 
    model=loaded_model, 
    threshold=0.5
)
prediction, probability